# Elaborate statistics features for Silvereye

This is a notebook to tease the statistics on AWRA and related data for Silvereye.

Merge capabilities from:

* Ben's code in the prototype app
* Ramneek's stat on daily data.
* possibly some of the princeton-chelsa blend code (optional)


Plan is:

* Subset AWRA daily data sets to a more manageable size for initial elaboration. ACT or Tassie.
* Get filthy rich and party what's left before Extinction

## Dependencies imports


In [ ]:
import xarray as xr
import os
import sys
import pandas as pd
from functools import wraps
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns  # noqa, pandas aware plotting library

In [ ]:
if ('SP_SRC' in os.environ):
    root_src_dir = os.environ['SP_SRC']
elif sys.platform == 'win32':
    root_src_dir = r'C:\src\csiro\stash\silverpieces'
else:
    root_src_dir = '/home/per202/src/csiro/stash/silverpieces'

pkg_src_dir = root_src_dir
sys.path.append(pkg_src_dir)

In [ ]:
#from silverpieces.blah import *

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:

from siphon.catalog import TDSCatalog
# from distributed import Client, LocalCluster
# from flask import Flask, redirect, url_for, request, render_template, jsonify, abort
# from flask_cors import CORS
# import intake
# import regionmask
# import random
# from flask import jsonify, make_response
# import json
# import uuid
# from flask.logging import default_handler
# import geopandas as gpd
# from owslib.wps import WebProcessingService
# from owslib.wps import printInputOutput
# from birdy import WPSClient
# import birdy
# import tarfile

# import tempfile
# import shutil
# import urllib
# import urllib.request
# import requests

# from urllib.parse import urlparse
# from datetime import datetime
# from dateutil.relativedelta import relativedelta


I tried to use [xarray-tips-and-tricks](https://rabernat.github.io/research_computing_2018/xarray-tips-and-tricks.html) by similarity but while this looked OK, retrieving values ended up in a `RuntimeError: NetCDF: Access failure`. Parking this approach:

In [ ]:
# #base_url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis/surface/air.sig995'
# base_url = 'http://data-mel.it.csiro.au/thredds/dodsC/catch_all/lw_oznome/AWRA-L_historical_data/qtot/qtot_avg_'
# f = ['http://data-mel.it.csiro.au/thredds/dodsC/catch_all/lw_oznome/AWRA-L_historical_data/qtot/qtot_avg_' + str(yr) + '.nc' for yr in [2011, 2012]]
# files = [f'{base_url}{year}.nc' for year in range(2010, 2015)]
# files

In [ ]:
# ds = xr.open_mfdataset(files)
# ds

In [ ]:
# x = ds.qtot_avg
# x[1,1,1].values

In [ ]:
cat_url = 'http://data-mel.it.csiro.au/thredds/catalog/catch_all/lw_oznome/AWRA-L_historical_data/qtot/catalog.xml'

In [ ]:
cat = TDSCatalog(cat_url)

In [ ]:
ds_names = [str(x) for x in cat.datasets]
m_indices = [i for i in range(len(ds_names)) if ds_names[i].startswith('qtot') and ds_names[i].endswith('nc')]
# somehow sorted
m_indices.reverse()

In [ ]:
ds = [cat.datasets[i] for i in m_indices]

In [ ]:
ds[0].access_urls

In [ ]:
dsets = [cds.remote_access(use_xarray=True).reset_coords(drop=True).chunk({'latitude': 681, 'longitude': 841})  # 731, latitude: 681, longitude: 841
         for cds in ds[:3]] # eventually want to use the whole catalog here
#dsets = [cds.remote_access(use_xarray=True).reset_coords(drop=True)
#         for cds in ds[:3]] # eventually want to use the whole catalog here

In [ ]:
len(dsets)

In [ ]:
blah = xr.auto_combine(dsets)
blah

In [ ]:
x = blah.qtot_avg

In [ ]:
x['time'].values

In [ ]:
# Random but frequently ends up RuntimeError: NetCDF: Access failure
m = x[1,:,:].values

In [ ]:
plt.imshow(m)

In [ ]:
%time b_box = x.isel(latitude=slice(600,700), longitude=slice(650,720))
%time b_box.isel(time=310).plot()
%time plt.show()